## PYSPARK ASSIGNMENT - GROUP 6

### Dataset : Entertainment - movie

i.Extract:  Load the data
   - Read data as csv via spark dataframe

ii.Transform: Exploratory data analysis using spark df
    - Unique id count
    - split column release_date into three columns -> year,month and day
    - Replace - month -> 1 to Jan and 2 to Feb and so on
    - Update : Normalize popularity column - normalized  v = v - min(popularity)/max(popularity)-min(popularity)
    - remove - overview and video column
    - show df
    - GroupBy vote_average and count

iii.Load: Save analysis report
    - show df, save as files


### I. Extract: Load the data

In [1]:
#Importing necessary libraries
import pandas as pd 
import os
import calendar
import itertools
from pyspark.sql import functions as f
from pyspark.sql.functions import *
from pyspark.sql.functions import countDistinct

In [2]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark.sql import SparkSession 
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType

In [3]:
#Creating a PySpark SparkSession
spark = SparkSession.builder.master("local").\
        appName("Python Spark SQL basic example").\
        config("spark.some.config.option", "some-value").\
        getOrCreate()

In [4]:
data = spark.read.format("com.databricks.spark.csv").option("header", True).option("multiline",True).option("escape", "\"").option("quote", '"').option("inferschema", True).option("delimiter",",").load("C:/Users/LATHA/Desktop/Movie-Dataset-Latest.csv")

In [5]:
#Spark-data : Final dataframe 
data.printSchema() #to see the schema of spark dataframe  
data.show()

root
 |-- _c0: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- video: boolean (nullable = true)

+---+------+--------------------+------------+--------------------+----------+------------+----------+-----+
|_c0|    id|               title|release_date|            overview|popularity|vote_average|vote_count|video|
+---+------+--------------------+------------+--------------------+----------+------------+----------+-----+
|  0| 19404|Dilwale Dulhania ...|  1995-10-20|Raj is a rich, ca...|    25.884|         8.7|      3304|false|
|  1|   278|The Shawshank Red...|  1994-09-23|Framed in the 194...|     60.11|         8.7|     20369|false|
|  2|   238|       The Godfather|  1972-03-14|Spanning the year...|    62.784|         8.7

In [6]:
data=data.drop(*['_c0'])

### ii.Transform: Exploratory data analysis using spark dataframe

#### To count unique id's

In [7]:
count=data.select(countDistinct("id"))
count.show()

+------------------+
|count(DISTINCT id)|
+------------------+
|              9463|
+------------------+



#### Splitting column release_date into three columns -> year,month and day

In [8]:
data=(data.withColumn("Year",f.year("release_date"))
          .withColumn("day",f.dayofmonth("release_date"))
          .withColumn("month",f.month("release_date")))

In [9]:
data.show()

+------+--------------------+------------+--------------------+----------+------------+----------+-----+----+---+-----+
|    id|               title|release_date|            overview|popularity|vote_average|vote_count|video|Year|day|month|
+------+--------------------+------------+--------------------+----------+------------+----------+-----+----+---+-----+
| 19404|Dilwale Dulhania ...|  1995-10-20|Raj is a rich, ca...|    25.884|         8.7|      3304|false|1995| 20|   10|
|   278|The Shawshank Red...|  1994-09-23|Framed in the 194...|     60.11|         8.7|     20369|false|1994| 23|    9|
|   238|       The Godfather|  1972-03-14|Spanning the year...|    62.784|         8.7|     15219|false|1972| 14|    3|
|724089|Gabriel's Inferno...|  2020-07-31|Professor Gabriel...|    28.316|         8.6|      1360|false|2020| 31|    7|
|   424|    Schindler's List|  1993-11-30|The true story of...|    38.661|         8.6|     12158|false|1993| 30|   11|
|696374|   Gabriel's Inferno|  2020-05-2

In [10]:
data.select(data.columns[2:5]).show(4)


+------------+--------------------+----------+
|release_date|            overview|popularity|
+------------+--------------------+----------+
|  1995-10-20|Raj is a rich, ca...|    25.884|
|  1994-09-23|Framed in the 194...|     60.11|
|  1972-03-14|Spanning the year...|    62.784|
|  2020-07-31|Professor Gabriel...|    28.316|
+------------+--------------------+----------+
only showing top 4 rows



#### Replace - month -> 1 to Jan and 2 to Feb and so on

In [11]:
months = f.create_map(*[
    f.lit(m) for m in itertools.chain(*[(x, calendar.month_abbr[x]) for x in range(1, 12, 1)])
])
data = data.withColumn("month", months[f.col("month")])
#data=dat.drop(*['month'])
data.show()

+------+--------------------+------------+--------------------+----------+------------+----------+-----+----+---+-----+
|    id|               title|release_date|            overview|popularity|vote_average|vote_count|video|Year|day|month|
+------+--------------------+------------+--------------------+----------+------------+----------+-----+----+---+-----+
| 19404|Dilwale Dulhania ...|  1995-10-20|Raj is a rich, ca...|    25.884|         8.7|      3304|false|1995| 20|  Oct|
|   278|The Shawshank Red...|  1994-09-23|Framed in the 194...|     60.11|         8.7|     20369|false|1994| 23|  Sep|
|   238|       The Godfather|  1972-03-14|Spanning the year...|    62.784|         8.7|     15219|false|1972| 14|  Mar|
|724089|Gabriel's Inferno...|  2020-07-31|Professor Gabriel...|    28.316|         8.6|      1360|false|2020| 31|  Jul|
|   424|    Schindler's List|  1993-11-30|The true story of...|    38.661|         8.6|     12158|false|1993| 30|  Nov|
|696374|   Gabriel's Inferno|  2020-05-2

#### Update - Normalizing popularity column

In [12]:
min = data.agg({"popularity": "min"}).collect()[0][0]
max = data.agg({"popularity": "max"}).collect()[0][0]

data = data.withColumn('popularity', (col('popularity') - min)/(max-min))

In [13]:
data.show()

+------+--------------------+------------+--------------------+--------------------+------------+----------+-----+----+---+-----+
|    id|               title|release_date|            overview|          popularity|vote_average|vote_count|video|Year|day|month|
+------+--------------------+------------+--------------------+--------------------+------------+----------+-----+----+---+-----+
| 19404|Dilwale Dulhania ...|  1995-10-20|Raj is a rich, ca...|0.001788613400169...|         8.7|      3304|false|1995| 20|  Oct|
|   278|The Shawshank Red...|  1994-09-23|Framed in the 194...|0.004209792099512666|         8.7|     20369|false|1994| 23|  Sep|
|   238|       The Godfather|  1972-03-14|Spanning the year...|0.004398953317360034|         8.7|     15219|false|1972| 14|  Mar|
|724089|Gabriel's Inferno...|  2020-07-31|Professor Gabriel...|0.001960655315578777|         8.6|      1360|false|2020| 31|  Jul|
|   424|    Schindler's List|  1993-11-30|The true story of...|0.002692470124341...|      

#### Remove - overview and video column

In [14]:
columns_to_drop = ['overview','video']
data=data.drop(*columns_to_drop)

In [15]:
data.show()

+------+--------------------+------------+--------------------+------------+----------+----+---+-----+
|    id|               title|release_date|          popularity|vote_average|vote_count|Year|day|month|
+------+--------------------+------------+--------------------+------------+----------+----+---+-----+
| 19404|Dilwale Dulhania ...|  1995-10-20|0.001788613400169...|         8.7|      3304|1995| 20|  Oct|
|   278|The Shawshank Red...|  1994-09-23|0.004209792099512666|         8.7|     20369|1994| 23|  Sep|
|   238|       The Godfather|  1972-03-14|0.004398953317360034|         8.7|     15219|1972| 14|  Mar|
|724089|Gabriel's Inferno...|  2020-07-31|0.001960655315578777|         8.6|      1360|2020| 31|  Jul|
|   424|    Schindler's List|  1993-11-30|0.002692470124341...|         8.6|     12158|1993| 30|  Nov|
|696374|   Gabriel's Inferno|  2020-05-29|0.001258834656542226|         8.6|      2172|2020| 29|  May|
|761053|Gabriel's Inferno...|  2020-11-19|0.002044058859274382|         8

##### GroupBy vote_average and count

In [16]:
f=data.groupBy("vote_average").count()

In [17]:
f.show()

+------------+-----+
|vote_average|count|
+------------+-----+
|         8.0|  101|
|         5.4|  145|
|         7.0|  383|
|         6.1|  405|
|         7.7|  201|
|         6.6|  444|
|         3.7|    6|
|         5.7|  248|
|         8.6|    8|
|         3.9|    8|
|         8.7|    3|
|         4.5|   28|
|         6.7|  441|
|         7.4|  350|
|         6.5|  432|
|         8.4|   38|
|         4.9|   50|
|         4.1|    8|
|         2.2|    1|
|         2.8|    1|
+------------+-----+
only showing top 20 rows



### iii.Load: Save analysis report
####  - show df, save as files

In [18]:
f.toPandas().to_csv("vote_avg_count.csv")


In [19]:
f1 = f.toPandas()
f1.to_csv('Output.txt', index=False)
print('Successfully saved at ',os.getcwd())

Successfully saved at  C:\Users\LATHA\Downloads


###   Submitted By:
 - Sarah Merin John   - 20BDA06 
 - Manu Tom           - 20BDA23
 - Keerthana Sajeevan - 20BDA39
 - Tonmoy Dutta       - 20BDA57
 - Ananya Kumari      - 20BDA68